In [1]:
import pandas as pd
import numpy as np

In [2]:
file = pd.read_csv("spam.csv",encoding="Windows-1252")

In [3]:
file = file.drop(["Unnamed: 3","Unnamed: 4","Unnamed: 2"],axis=1)

In [4]:
file.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
file.rename(columns={"v1":"class_label","v2":"message"},inplace=True)

In [6]:
file.tail()

,class_label,message
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...
5571,ham,Rofl. Its true to its name


In [7]:
file['class_label'].value_counts()

ham     4825
spam     747
Name: class_label, dtype: int64

In [8]:
## OK
df=file

In [9]:
df_spam = file[file.class_label=='spam']

In [10]:
spam_list= df_spam['message'].tolist()

In [11]:
filtered_spam = "".join(spam_list).lower()

In [12]:
filtered_spam

'free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005. text fa to 87121 to receive entry question(std txt rate)t&c\'s apply 08452810075over18\'sfreemsg hey there darling it\'s been 3 week\'s now and no word back! i\'d like some fun you up for it still? tb ok! xxx std chgs to send, å£1.50 to rcvwinner!! as a valued network customer you have been selected to receivea å£900 prize reward! to claim call 09061701461. claim code kl341. valid 12 hours only.had your mobile 11 months or more? u r entitled to update to the latest colour mobiles with camera for free! call the mobile update co free on 08002986030six chances to win cash! from 100 to 20,000 pounds txt> csh11 and send to 87575. cost 150p/day, 6days, 16+ tsandcs apply reply hl 4 infourgent! you have won a 1 week free membership in our å£100,000 prize jackpot! txt the word: claim to no: 81010 t&c www.dbuk.net lccltd pobox 4403ldnw1a7rw18xxxmobilemovieclub: to use your credit, click the wap link in the next txt message or c

In [13]:
df['class_label'] = df['class_label'].apply(lambda x: 1 if x == 'spam' else 0)

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
x_train, x_test, y_train, y_test = train_test_split(df['message'], df['class_label'], test_size = 0.2, random_state = 42)

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [17]:
print('rows in test set: ' + str(x_test.shape))
print('rows in train set: ' + str(x_train.shape))

rows in test set: (1115,)
rows in train set: (4457,)


In [18]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
vect = TfidfVectorizer(max_features = 1000, stop_words='english')
#vect = TfidfVectorizer(stop_words='english')
print(vect)
pickle.dump(vect, open("saved_models/vectorizer.pickle", "wb"))

TfidfVectorizer(max_features=1000, stop_words='english')


In [19]:
vect = pickle.load(open("saved_models/vectorizer.pickle", 'rb'))

In [20]:
x_train_dtm = vect.fit_transform(x_train)
pickle.dump(vect, open("saved_models/vectorizer.pickle", "wb"))

In [21]:
vect = pickle.load(open("saved_models/vectorizer.pickle", 'rb'))
x_test_dtm = vect.transform(x_test)

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
logreg = LogisticRegression(C=1, max_iter = 2000)
NB = MultinomialNB()
import pickle



# train the model using X_train_dtm & y_train
logreg.fit(x_train_dtm, y_train)
filename = "saved_models/lr_model.pkl"
pickle.dump(logreg, open(filename, 'wb'))
# compute the training accuracy
y_pred_train = logreg.predict(x_train_dtm)
print('Training accuracy is {}'.format(accuracy_score(y_train, y_pred_train)))
# compute the predicted probabilities for X_test_dtm
y_pred_test = logreg.predict(x_test_dtm)
print('Test accuracy is {}'.format(accuracy_score(y_test,y_pred_test)))
print(confusion_matrix(y_test,y_pred_test))

Training accuracy is 0.9782364819385236
Test accuracy is 0.967713004484305
[[962   3]
 [ 33 117]]


In [23]:
import pickle
import sklearn
vect = pickle.load(open("saved_models/vectorizer.pickle", 'rb'))
model = pickle.load(open("saved_models/lr_model.pkl","rb"))

In [24]:
def prediction(string):
    string=[string]
    sen_trans = vect.transform(string)
    p = model.predict(sen_trans)
    return ["ham","spam"][p[0]]


In [25]:
test1 = file["message"][2]

In [26]:
test2 = file["message"][0]

In [27]:
print("Test text : {}".format(test1))
print()
print("Prediction : {}".format(prediction(test1)))

Test text : Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's

Prediction : spam


In [28]:
print("Test text : {}".format(test2))
print()
print("Prediction : {}".format(prediction(test2)))

Test text : Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...

Prediction : ham
